This notebook is for converting a Keras Model to a TensorFlow Model

## Setup

In [1]:
%%bash

echo 'Train:'
TRAIN_DIR=../../../tmp/trt_end_to_end/train
find ${TRAIN_DIR} -type f | sort
echo
echo 'Convert (Saved Graph):'
CONVERT_TF_SAVED_GRAPH_DIR=../../../tmp/trt_end_to_end/convert/tf/saved
find ${CONVERT_TF_SAVED_GRAPH_DIR} -type f | sort

Train:
../../../tmp/trt_end_to_end/train/basic/001/basic_epoch001_2019-09-03T19:15.h5
../../../tmp/trt_end_to_end/train/basic/001/basic_epoch001_2019-09-03T19:15.md
../../../tmp/trt_end_to_end/train/basic/001/params.json
../../../tmp/trt_end_to_end/train/batchn/001/batchn_epoch001_2019-09-03T19:28.h5
../../../tmp/trt_end_to_end/train/batchn/001/batchn_epoch001_2019-09-03T19:28.md
../../../tmp/trt_end_to_end/train/batchn/001/params.json
../../../tmp/trt_end_to_end/train/conv/001/conv_epoch001_2019-09-03T19:30.h5
../../../tmp/trt_end_to_end/train/conv/001/conv_epoch001_2019-09-03T19:30.md
../../../tmp/trt_end_to_end/train/conv/001/params.json
../../../tmp/trt_end_to_end/train/resnet50/001/params.json
../../../tmp/trt_end_to_end/train/resnet50/001/resnet50_epoch001_2019-09-03T19:31.h5
../../../tmp/trt_end_to_end/train/resnet50/001/resnet50_epoch001_2019-09-03T19:31.md

Convert (Saved Graph):
../../../tmp/trt_end_to_end/convert/tf/saved/basic/001/checkpoint
../../../tmp/trt_end_to_end/conv

In [2]:
import sys
sys.path.append('../../..')

## Parameters

In [3]:
_NAME = 'resnet50'
_EPOCH = 1

In [4]:
from src.utils.trt_end_to_end_constants import *
_NAME, _EPOCH, _TIME = get_params(_NAME, _EPOCH)

In [5]:
_train_filename = H5_FILE_FORMAT % (_NAME, _EPOCH, _TIME)
_train_save_to_dir = get_train_dir(_NAME, _EPOCH)
_train_filepath = os.path.join(_train_save_to_dir, _train_filename)
assert os.path.exists(_train_filepath), "Not exists: %s" % _train_filepath

## Set learning phase 0

In [6]:
# import the needed libraries
import tensorflow as tf
print(tf.__name__, tf.__version__, sep='-')
tf.keras.backend.set_learning_phase(0) # use this if we have batch norm layer in our network

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

tensorflow-1.14.0


/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

## Load keras model

In [7]:
model_filename = H5_FILE_FORMAT % (_NAME, _EPOCH, _TIME)
model_filepath = os.path.join(_train_save_to_dir, model_filename)
print(model_filepath)

keras_model = tf.keras.models.load_model(model_filepath)

W0903 19:33:35.543020 140236397193024 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0903 19:33:35.543934 140236397193024 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0903 19:33:35.546770 140236397193024 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprec

../../../tmp/trt_end_to_end/train/resnet50/001/resnet50_epoch001_2019-09-03T19:31.h5


W0903 19:33:37.574040 140236397193024 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


## Save tensorflow model

In [8]:
%%bash

CONVERT_TF_SAVED_GRAPH_DIR=../../../tmp/trt_end_to_end/convert/tf/saved
mkdir -p ${CONVERT_TF_SAVED_GRAPH_DIR}
#rm -f ./${CONVERT_TF_DIR}/*
find ${CONVERT_TF_SAVED_GRAPH_DIR} -type f | sort

../../../tmp/trt_end_to_end/convert/tf/saved/conv/001/tf_model_conv_epoch001_2019-09-03T19:30.data-00000-of-00001
../../../tmp/trt_end_to_end/convert/tf/saved/conv/001/tf_model_conv_epoch001_2019-09-03T19:30.meta
../../../tmp/trt_end_to_end/convert/tf/saved/conv/001/checkpoint
../../../tmp/trt_end_to_end/convert/tf/saved/conv/001/tf_model_conv_epoch001_2019-09-03T19:30.index
../../../tmp/trt_end_to_end/convert/tf/saved/basic/001/tf_model_basic_epoch001_2019-09-03T19:15.index
../../../tmp/trt_end_to_end/convert/tf/saved/basic/001/checkpoint
../../../tmp/trt_end_to_end/convert/tf/saved/basic/001/tf_model_basic_epoch001_2019-09-03T19:15.data-00000-of-00001
../../../tmp/trt_end_to_end/convert/tf/saved/basic/001/tf_model_basic_epoch001_2019-09-03T19:15.meta
../../../tmp/trt_end_to_end/convert/tf/saved/batchn/001/tf_model_batchn_epoch001_2019-09-03T19:28.data-00000-of-00001
../../../tmp/trt_end_to_end/convert/tf/saved/batchn/001/tf_model_batchn_epoch001_2019-09-03T19:28.meta
../../../tmp/trt

In [9]:
file_prefix = SAVED_GRAPH_FILE_FORMAT % (_NAME, _EPOCH ,_TIME)
save_to_dir = get_saved_graph_dir(_NAME, _EPOCH)
save_to = os.path.join(save_to_dir, file_prefix)

# save the model to Tensorflow model
saver = tf.train.Saver()
sess = tf.keras.backend.get_session()
save_path = saver.save(sess, save_to)

print("Keras model is successfully converted to TF model in", save_to_dir)

Keras model is successfully converted to TF model in ../../../tmp/trt_end_to_end/convert/tf/saved/resnet50/001


In [10]:
%%bash

CONVERT_TF_SAVED_GRAPH_DIR=../../../tmp/trt_end_to_end/convert/tf/saved
find ${CONVERT_TF_SAVED_GRAPH_DIR} -type f | sort

../../../tmp/trt_end_to_end/convert/tf/saved/basic/001/checkpoint
../../../tmp/trt_end_to_end/convert/tf/saved/basic/001/tf_model_basic_epoch001_2019-09-03T19:15.data-00000-of-00001
../../../tmp/trt_end_to_end/convert/tf/saved/basic/001/tf_model_basic_epoch001_2019-09-03T19:15.index
../../../tmp/trt_end_to_end/convert/tf/saved/basic/001/tf_model_basic_epoch001_2019-09-03T19:15.meta
../../../tmp/trt_end_to_end/convert/tf/saved/batchn/001/checkpoint
../../../tmp/trt_end_to_end/convert/tf/saved/batchn/001/tf_model_batchn_epoch001_2019-09-03T19:28.data-00000-of-00001
../../../tmp/trt_end_to_end/convert/tf/saved/batchn/001/tf_model_batchn_epoch001_2019-09-03T19:28.index
../../../tmp/trt_end_to_end/convert/tf/saved/batchn/001/tf_model_batchn_epoch001_2019-09-03T19:28.meta
../../../tmp/trt_end_to_end/convert/tf/saved/conv/001/checkpoint
../../../tmp/trt_end_to_end/convert/tf/saved/conv/001/tf_model_conv_epoch001_2019-09-03T19:30.data-00000-of-00001
../../../tmp/trt_end_to_end/convert/tf/saved